In [1]:
## 导入库
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools
import random
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing
from pathlib import Path
import os.path
from sklearn.metrics import classification_report

In [2]:
BATCH_SIZE = 1024 ## 设置超参数
TARGET_SIZE = (224, 224)

In [3]:
dataset = "./data/Image Data base"

In [4]:
def readimg(dataset): ## 读取图片
    image_dir = Path(dataset)
    filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.PNG'))

    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    image_df = pd.concat([filepaths, labels], axis=1)
    return image_df

image_df = readimg(dataset)

In [5]:
train_df, test_df = train_test_split(image_df, test_size=0.3, shuffle=True, random_state=10)## 按照训练集和测试集7:3分数据


In [6]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.25
)## 生成训练集数据

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)## 生成测试集数据

In [7]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)## 生成训练集图像数据

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)## 生成验证集图像数据

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)# 生成测试集图像数据

Found 209649 validated image filenames belonging to 58 classes.
Found 69882 validated image filenames belonging to 58 classes.
Found 119799 validated image filenames belonging to 58 classes.


In [8]:
augment = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomFlip("vertical"),
])## 设置模型的数据变换参数

In [9]:
pretrained_model = tf.keras.applications.efficientnet.EfficientNetB7(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)## 设置EfficientNetB7模型

pretrained_model.trainable = False

In [10]:
checkpoint_path = "pests_cats_classification_model_checkpoint"## 设置保存模型的ModelCheckpoint
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

In [11]:
## 设置提前停止训练的参数
early_stopping = EarlyStopping(monitor = "val_loss",
                               patience = 5,
                               restore_best_weights = True)

In [12]:
import datetime
## 创建保存模型用的模型callback
def create_tensorboard_callback(dir_name, experiment_name):

    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

In [15]:
# 建立模型包括：模型输入，模型结构，模型的损失函数，模型的训练参数
inputs = pretrained_model.input
x = augment(inputs)

x = Dense(128, activation='relu')(pretrained_model.output)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)


outputs = Dense(58, activation='softmax')(x)

x = tf

model = Model(inputs=inputs, outputs=outputs)
## 编译模型
model.compile(
    optimizer=Adam(0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
## 训练模型
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=100,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs",
                                    "pests_cats_classification"),
        checkpoint_callback,
    ]
)

Saving TensorBoard log files to: training_logs/pests_cats_classification/20230503-135709
Epoch 1/100
6552/6552 [==============================] - 2552s 388ms/step - loss: 2.6815 - accuracy: 0.3655 - val_loss: 1.7940 - val_accuracy: 0.5530
Epoch 2/100
6552/6552 [==============================] - 2470s 377ms/step - loss: 1.8953 - accuracy: 0.5096 - val_loss: 1.3234 - val_accuracy: 0.6493
Epoch 3/100
6552/6552 [==============================] - 2424s 370ms/step - loss: 1.5665 - accuracy: 0.5746 - val_loss: 1.0520 - val_accuracy: 0.7242
Epoch 4/100
6552/6552 [==============================] - 2378s 363ms/step - loss: 1.3443 - accuracy: 0.6241 - val_loss: 0.8538 - val_accuracy: 0.7726
Epoch 5/100
6552/6552 [==============================] - 2378s 363ms/step - loss: 1.1779 - accuracy: 0.6646 - val_loss: 0.7162 - val_accuracy: 0.8117
Epoch 6/100
6552/6552 [==============================] - 2469s 377ms/step - loss: 1.0543 - accuracy: 0.6963 - val_loss: 0.6171 - val_accuracy: 0.8375
Epoch 7/100

In [18]:
results = model.evaluate(test_images, verbose=0)
## 打印测试集准确率
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

In [19]:
## 绘制训练过程测试集和训练集的准确率和误差变化图
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')

plt.title('Training and validation loss')
plt.legend()
plt.show()

In [20]:
### 对测试集进行预测，展示5个结果
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

print(f'The first 5 predictions: {pred[:10]}')

In [21]:
## 展示分类结果
random_index = np.random.randint(0, len(test_df) - 1, 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_index[i]]))
    if test_df.Label.iloc[random_index[i]] == pred[random_index[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}", color=color)
plt.show()
plt.tight_layout()

In [22]:
## 打印测试集预测结果
y_test = list(test_df.Label)
print(classification_report(y_test, pred))